<a href="https://colab.research.google.com/github/Tlen33/Text-processing-with-neural-networks/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
from google.colab import files
import time 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
# Подключение диска
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Создаём функцию для чтения файла. На вход отправляем путь к файлу
def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  text = text.replace("\n", " ")
  return text

className = ["О. Генри", "Стругацкие", "Булгаков", "Клиффорд", "Макс", "Брэдберри"]
nClasses = len(className)

In [ ]:
# Загружаем обучающие тексты
trainText = []
testText = []

for i in className:
  for j in os.listdir('/content/drive/MyDrive/Colab Notebooks/Базы/Тексты писателей/'):
    if i in j:
      if 'Обучающая' in j: 
        trainText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Тексты писателей/' + j))
        print(j, 'добавлен в обучающую выборку')
      if 'Тестовая' in j:
        testText.append(readText('/content/drive/MyDrive/Colab Notebooks/Базы/Тексты писателей/' + j))
        print(j, 'добавлен в тестовую выборку')
  print()

(О. Генри) Тестовая_20 вместе.txt добавлен в тестовую выборку
(О. Генри) Обучающая_50 вместе.txt добавлен в обучающую выборку

(Стругацкие) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Стругацкие) Тестовая_2 вместе.txt добавлен в тестовую выборку

(Булгаков) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Булгаков) Тестовая_2 вместе.txt добавлен в тестовую выборку

(Клиффорд_Саймак) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Клиффорд_Саймак) Обучающая_5 вместе.txt добавлен в обучающую выборку

(Макс Фрай) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Макс Фрай) Обучающая_5 вместе.txt добавлен в обучающую выборку

(Рэй Брэдберри) Обучающая_22 вместе.txt добавлен в обучающую выборку
(Рэй Брэдберри) Тестовая_8 вместе.txt добавлен в тестовую выборку



In [ ]:
# Проверяем количество элементов и символов
print(len(trainText))
print(len(trainText[0]))

6
1049517


In [ ]:
# Обработка данных
cur_time = time.time()
maxWordsCount = 20000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(trainText)
items = list(tokenizer.word_index.items())
print('Время обработки: ', round(time.time() - cur_time, 2), 'c', sep='')
print('50 самых часто встречающихся слов:\n', items[:50])
print("Размер словаря:", len(items))

Время обработки: 3.74c
50 самых часто встречающихся слов:
 [('unknown', 1), ('и', 2), ('в', 3), ('не', 4), ('я', 5), ('что', 6), ('на', 7), ('с', 8), ('он', 9), ('а', 10), ('как', 11), ('то', 12), ('это', 13), ('но', 14), ('все', 15), ('у', 16), ('по', 17), ('его', 18), ('к', 19), ('так', 20), ('мне', 21), ('из', 22), ('за', 23), ('меня', 24), ('ты', 25), ('же', 26), ('бы', 27), ('сказал', 28), ('вы', 29), ('было', 30), ('от', 31), ('они', 32), ('мы', 33), ('только', 34), ('да', 35), ('еще', 36), ('она', 37), ('о', 38), ('вот', 39), ('когда', 40), ('если', 41), ('уже', 42), ('был', 43), ('нет', 44), ('ни', 45), ('их', 46), ('ну', 47), ('чтобы', 48), ('до', 49), ('для', 50)]
Размер словаря: 133070


In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)

In [ ]:
# Формирование обучающей выборки по листу индексов слов
def getSetFromIndexes(wordIndexes, xLen, step):
  xSample = []
  wordsLen = len(wordIndexes)
  index = 0

  while (index + xLen <= wordsLen):
    xSample.append(wordIndexes[index:index+xLen])
    index += step

  return xSample

In [ ]:
# Формирование обучающей и проверочной выборки
def createSetsMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)
  classesXSamples = []
  for wI in wordIndexes:
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)):
      xSamples.append(xT[i])
      ySamples.append(utils.to_categorical(t, nClasses))

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [ ]:
# Формируем обучающую и тестовую выборку и преобразуем в матрицы
xLen = 1000
step = 100
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)
xTrain01 = tokenizer.sequences_to_matrix(xTrain.tolist())
xTest01 = tokenizer.sequences_to_matrix(xTest.tolist())

In [ ]:
# Представляем тестовую выборку в удобных для распознавания размерах
def createTestMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)
  xTestClasses01 = []
  xTestClasses = []
  for wI in wordIndexes:
    sample = (getSetFromIndexes(wI, xLen, step))
    xTestClasses.append(sample)
    xTestClasses01.append(tokenizer.sequences_to_matrix(sample))
  xTestClasses01 = np.array(xTestClasses01)
  xTestClasses = np.array(xTestClasses)
  
  return xTestClasses01, xTestClasses

def recognizeMultiClass(model, xTest, modelName):
  print("НЕЙРОНКА: ", modelName)
  print()
  
  totalSumRec = 0
  
  for i in range(nClasses):
    currPred = model.predict(xTest[i])
    currOut = np.argmax(currPred, axis=1)
    evVal = []
    for j in range(nClasses):
      evVal.append(len(currOut[currOut==j])/len(xTest[i]))
    totalSumRec += len(currOut[currOut==i])
    recognizedClass = np.argmax(evVal)
    isRecognized = "Это НЕПРАВИЛЬНЫЙ ответ!"
    if (recognizedClass == i):
      isRecognized = "Это ПРАВИЛЬНЫЙ ответ!"
    str1 = 'Класс: ' + className[i] + " " * (11 - len(className[i])) + str(int(100*evVal[i])) + "% сеть отнесла к классу " + className[recognizedClass]
    print(str1, " " * (55-len(str1)), isRecognized, sep='')
  
  print()
  sumCount = 0
  for i in range(nClasses):
    sumCount += len(xTest[i])
  print("Средний процент распознавания: ", int(100*totalSumRec/sumCount), "%", sep='')
  
  return totalSumRec/sumCount

In [ ]:
xTestClasses01, x2 = createTestMultiClasses(testWordIndexes, xLen, step)
l = np.array(xTestClasses01)
np.save('xTestPredictBoW', l)
np.save('xTestPredictEmbedding', x2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Создаём полносвязную сеть
model = Sequential()
model.add(Dense(200, input_dim=maxWordsCount, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(6, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(xTrain01, yTrain, epochs=10, batch_size=128, validation_data=(xTest01, yTest))

Epoch 1/10
138/138 [==============================] - 14s 94ms/step - loss: 0.0455 - accuracy: 0.9857 - val_loss: 0.4033 - val_accuracy: 0.8818
Epoch 2/10
138/138 [==============================] - 10s 75ms/step - loss: 3.1500e-04 - accuracy: 1.0000 - val_loss: 0.3504 - val_accuracy: 0.8841
Epoch 3/10
138/138 [==============================] - 10s 75ms/step - loss: 1.6298e-04 - accuracy: 1.0000 - val_loss: 0.3414 - val_accuracy: 0.8830
Epoch 4/10
138/138 [==============================] - 12s 84ms/step - loss: 1.0044e-04 - accuracy: 1.0000 - val_loss: 0.3410 - val_accuracy: 0.8803
Epoch 5/10
138/138 [==============================] - 10s 75ms/step - loss: 7.6769e-05 - accuracy: 1.0000 - val_loss: 0.3361 - val_accuracy: 0.8830
Epoch 6/10
138/138 [==============================] - 11s 78ms/step - loss: 5.6834e-05 - accuracy: 1.0000 - val_loss: 0.3344 - val_accuracy: 0.8824
Epoch 7/10
138/138 [==============================] - 10s 74ms/step - loss: 4.7518e-05 - accuracy: 1.0000 - val_loss

In [ ]:
# Проверяем точность нейронки обученной на bag of words
pred = recognizeMultiClass(model, xTestClasses01, "нейросеть по классификации текстов писателей")

НЕЙРОНКА:  нейросеть по классификации текстов писателей

Класс: О. Генри   95% сеть отнесла к классу О. Генри   Это ПРАВИЛЬНЫЙ ответ!
Класс: Стругацкие 87% сеть отнесла к классу Стругацкие Это ПРАВИЛЬНЫЙ ответ!
Класс: Булгаков   71% сеть отнесла к классу Булгаков   Это ПРАВИЛЬНЫЙ ответ!
Класс: Клиффорд   87% сеть отнесла к классу Клиффорд   Это ПРАВИЛЬНЫЙ ответ!
Класс: Макс       91% сеть отнесла к классу Макс       Это ПРАВИЛЬНЫЙ ответ!
Класс: Брэдберри  98% сеть отнесла к классу Брэдберри  Это ПРАВИЛЬНЫЙ ответ!

Средний процент распознавания: 88%
